In [46]:
%cd ..

C:\Users\Elvira\Documents\python_stuff\projects


In [47]:
import numpy as np
from scipy.ndimage import convolve
from tqdm import tqdm_notebook as tqdm

from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from  ipywidgets import interact
output_notebook()

from recaps.utils import euler_forward, runge_kutta2, convert2img

Loading BokehJS ...

Let's implement the system described by [Guin and Baek, 2018](https://www.sciencedirect.com/science/article/abs/pii/S0378475417303531). It is a also predator-prey system, but with nonlinearities and some general "bells and whistles". In this system predator-prey encounters follow [Michaelis-Menten](https://en.wikipedia.org/wiki/Michaelis%E2%80%93Menten_kinetics) kinetics $const \frac{x_{prey}}{K_{prey} + x_{prey}}x_{pred}$. 

> <font size=2>Michaelis-Menten term is just a soft mathematical switch, which reflects simple logic:
    <br>- when prey "concentration" is higher than threshold $K_{prey}$, predator does not experience any food scarcity, and its growth rate mostly depends on own "concentration" (approximately proportional to $x_{pred}$");
    <br>- when prey "concentration" is lower than threshold $K_{prey}$, predator's wellbeing _does_ depend on the amount of available prey, so its growth rate depends on "concentrations" of both its own species _and_ its prey  (approximately proportional to $x_{prey}x_{pred}$).
     </font>

Additionally predators die out "due to old age", with rate proportional to their "concentration"; and due to intraspecific competition, with rate proportional to probability of two predators being at the same time at the same place. As for prey, it is assumed to grow on some mysterious food source, with rate proportional to prey's concentration. Although, since the food source is not infinite, the growth of prey is additionally limited by the carrying capacity of the environment $K_{cap}$. Apart form that, prey is continuously harvested, with the rate proportional to its "concentration". Additionally, both predator and prey can diffuse through environment. Therefore we can write:

$$ \begin{cases} 
   \frac{dx_{prey}}{dt} = D_{prey}\nabla^2 x_{prey} + r x_{prey}\big(1 - \frac{x_{prey}}{K_{cap}}\big) - \frac{\mu_{pred}}{Y} \frac{x_{prey}}{K_{prey} + x_{prey}}x_{pred} - H x_{prey} \\
   \frac{dx_{pred}}{dt} = D_{pred}\nabla^2 x_{pred} - b_{pred} x_{pred} + \mu_{pred} \frac{x_{prey}}{a_2 + x_{prey}}x_{pred} - c_{pred} x_{pred}^2 
   \end{cases}$$
   
where  

<table>
<tr><td> __Symbol__  </td> <td> __Value__ </td> <td> __Interpretation__ </td> </tr>
<tr><td> $D_{prey}$  </td> <td> 0.1  </td> <td> Diffusion constant of prey </td> </tr>
<tr><td> $D_{pred}$  </td> <td> 20   </td> <td> Diffusion constant of predator </td> </tr>
<tr><td> $r$         </td> <td> 1.0  </td> <td> Specific growth rate of prey </td> </tr>
<tr><td> $K_{cap}$   </td> <td> 1.0  </td> <td> Carrying capacity of the environment </td> </tr>
<tr><td> $Y$         </td> <td> 1.0  </td> <td> Predator yield on prey </td> </tr>
<tr><td> $K_{prey}$  </td> <td> 0.3  </td> <td> Predator half-saturation constant on prey </td> </tr>
<tr><td> $H$         </td> <td> 0.1  </td> <td> Prey harvest rate </td> </tr>
<tr><td> $b_{pred}$  </td> <td> 0.02 </td> <td> Predator mortality rate </td> </tr>
<tr><td> $\mu_{pred}$</td> <td> 1.0  </td> <td> Predator growth rate on prey</td> </tr>
<tr><td> $c_{pred}$  </td> <td> 0.5 - 1.0 </td> <td> Predator mortality rate due to intraspecific competition</td> </tr>
</table>

To implement this system, we can just copy-paste the code, that we already wrote, and modify the reaction term (both stoichiometry and kinetics).


In [50]:
def pred_prey(_, x, p):
    x = x.reshape(p['size'])   # resize flat 1D array back into (height,width,#channels)-array
    dxdt = np.zeros(p['size']) # preallocate the accumulation terms    
    reaction  = np.zeros(p['size']) # preallocate the reaction term
    diffusion = np.zeros(p['size']) # preallocate the diffusion term
    
    prey, pred = x[:,:,0], x[:,:,1]
    rates = np.stack((p['r_prey'] * prey * (1 - prey/p['K_cap']),      # growth of prey
                      p['b_prey'] * prey,                              # harvesting of prey
                      p['mu_pred'] * prey/(p['K_prey'] + prey) * pred, # growth of predator
                      p['b_pred'] * pred,                              # death of predator
                      p['c_pred'] * pred**2),                          # death of predator (competition) 
                      axis=2)
    
    laplacian = np.array([[0,  1, 0], 
                          [1, -4, 1], 
                          [0,  1, 0]])   
    
    for k in range(p['size'][2]):
        # --- get diffusion term ---        
        diffusion[:,:,k] = p['D'][k]/p['h']**2 * convolve(x[:,:,k], laplacian, mode="nearest")
        
        # --- get reaction term ---
        reaction[:,:,k] = np.sum(p['S'][:,k].reshape(1,1,-1)*rates, axis=2)
        
    # --- get total accumulation term ---
    dxdt = diffusion + reaction
        
    return dxdt.ravel()


# parameters: system 
p = {'mu_pred':1,    # predator growth rate on prey (attack rate) (1)
     'Y'      :1,    # predator yield on prey (1)
     'K_prey' :0.3,  # predator half-saturation coeff on prey (0.3 -> stable; 0.1265 -> bifurcation[@c_pred=0.5])
     'b_pred' :0.02, # predator mortality rate (0.02)
     'c_pred' :0.5,  # predator mortality rate due to competition (0.5 -> dots; 1.0 -> stripes)
     'r_prey' :1,    # prey growth rate (1)
     'K_cap'  :1,    # carrying capacity for prey (1)
     'b_prey' :0.1,  # prey mortality/harvest rate (0.1)
     'D'      :[0.1, 20]} # diffusion rates for prey and pred

# parameters: spatial grid
resolution = [75,75] # grid dimensions in pixels
p['size'] = (*resolution, 2) # (height, width, #states)
p['h'] = 1 # scale: pixel size in physical units

# stoichiometry mtx:      prey  pred
p['S'] = np.array([[        1,    0], # growth of prey
                   [       -1,    0], # harvesting of prey
                   [-1/p['Y'],    1], # growth of predator
                   [        0,   -1], # death of predator
                   [        0,   -1]])# death of predator due to competition

# time-related
t0, tf, dt = 0, 150, 0.01
t_span = np.arange(t0, tf+dt, dt)

# initial condition
x0 = 0.5*np.random.rand(*p['size']).ravel()

# run the simulation: use Euler Forward to find the RGB "concentrations" numerically
img_development = euler_forward(pred_prey, x0, t_span, p)


HBox(children=(IntProgress(value=0, max=15000), HTML(value='')))

In [51]:
# plot stuff!
img_data = ColumnDataSource({'img': [np.flipud(convert2img(img_development.T[0], p['size']))]})

pb = figure(x_range=(0,p['size'][0]), y_range=(0,p['size'][1]),
            plot_width=p['size'][0]*4, plot_height=p['size'][1]*4)
r = pb.image_rgba(image='img', source=img_data, x=0, y=0, dw=p['size'][0], dh=p['size'][1])
show(pb, notebook_handle=True)  

def update(t=0):
    i = int(t/dt)
    # update data_source
    #(adjust the contrast to make patterns more visible)
    r.data_source.data['img'] = [np.flipud(convert2img(img_development.T[i], p['size'], 
                                                       steepness=-10, midpoint=0.1))]
    push_notebook()
interact(update, t=(t0,tf,dt))

interactive(children=(FloatSlider(value=0.0, description='t', max=150.0, step=0.01), Output()), _dom_classes=('widget-interact',))

<function __main__.update>

Try out different parameter values for $c_{pred}$ and check how it affects the patterns formed! So far we haven't talked about the effect of parameters at all. That's mainly because it's a huge topic on its own, and it definitely deserves [its own notebook](.\..\parameters2behaviour.ipynb).